In [1]:
import pandas as pd
from tqdm import tqdm
import copy
import itertools
import random
import datetime
import helper

In [2]:
def sampleFromDict(d, sample):
    keys = random.sample(list(d), sample)
    values = [d[k] for k in keys]
    return dict(zip(keys, values))

In [3]:
def test(ivDepth, rk, sampleCounts, randomSeed = 4010):
    filename = helper.intArray2HexString(rk)+'_'+str(ivDepth*ivDepth*ivDepth)+'.csv'

    outputs = {}
    random.seed(randomSeed)
    iv0Range = random.sample([i for i in range(256)], ivDepth)
    random.seed(randomSeed)
    iv1Range = random.sample([i for i in range(256)], ivDepth)
    random.seed(randomSeed)
    iv2Range = random.sample([i for i in range(256)], ivDepth)

    for iv0 in tqdm(iv0Range):
        for iv1 in iv1Range:
            for iv2 in iv2Range:
                key = [iv0,iv1,iv2]+rk
                rc4Output = helper.rc4(key)
                outputs[(iv0,iv1,iv2)] = rc4Output[:256]

    keyPositions = [0,1,2,3,4]
    globalResults = {}
    for sampleCount in sampleCounts:
        print('running ' + str(sampleCount) + " at " + str(datetime.datetime.now().strftime("%H:%M:%S")))
        sampleOutputs = sampleFromDict(outputs, sampleCount)
        results = {k:[] for k in keyPositions}
        for i in keyPositions:
            for iv, ks in sampleOutputs.items():
                S = [i for i in range(0, 256)]
                throwaway, Ss, Js = helper.KSA(S, helper.rc4KeyPrep(list(iv), S), 4)
                idx = (3+i-ks[2+i])%256
                a = Ss[3].index(idx)
                b = 0
                for xx in range(3, 3+i+1):
                    b = b + Ss[3][xx]
                summ = (a-(Js[3]+b))%256
                results[i].append(summ)
        resultsRanking = {}
        for k in results.keys():
            #getTop = 256
            dff = pd.DataFrame(results[k], columns = ['x'])['x'].value_counts().reset_index()
            tpl = {tuple(r)[0]: tuple(r)[1] for r in dff.to_numpy().tolist()}
            resultsRanking[k] = tpl
        globalResults[sampleCount] = resultsRanking
    tablePrep = []
    for sampleCount, result in globalResults.items():
        for keyByte, ranking in result.items():
            for byteValue, count in ranking.items():
                tablePrep.append((sampleCount, keyByte, byteValue, count))
    resultDf = pd.DataFrame(tablePrep, columns = ['sampleCount', 'keyByte', 'byteValue', 'count'])
    resultDf.to_csv(filename)
    return filename

In [4]:
ivDepth = 100
rks = [
    #helper.hexString2IntArray('1122334455'),
    helper.hexString2IntArray('6677889900'),
    #helper.hexString2IntArray('FEDCBA0987'),
    #helper.hexString2IntArray('5B5FE58A81'),
    #helper.hexString2IntArray('010203FEDC'), #DONE
    
]
sampleCounts = [x*500 for x in range(1,51)]+[x*2000 for x in range(13,26)]+[x*2500 for x in range(21,41)]
for rk in rks:
    test(ivDepth, rk, sampleCounts)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:03<00:00,  2.43s/it]


running 500 at 11:26:22
running 1000 at 11:26:22
running 1500 at 11:26:23
running 2000 at 11:26:24
running 2500 at 11:26:25
running 3000 at 11:26:27
running 3500 at 11:26:29
running 4000 at 11:26:31
running 4500 at 11:26:34
running 5000 at 11:26:37
running 5500 at 11:26:40
running 6000 at 11:26:44
running 6500 at 11:26:47
running 7000 at 11:26:51
running 7500 at 11:26:55
running 8000 at 11:27:00
running 8500 at 11:27:05
running 9000 at 11:27:10
running 9500 at 11:27:15
running 10000 at 11:27:21
running 10500 at 11:27:27
running 11000 at 11:27:34
running 11500 at 11:27:40
running 12000 at 11:27:48
running 12500 at 11:27:55
running 13000 at 11:28:03
running 13500 at 11:28:11
running 14000 at 11:28:20
running 14500 at 11:28:30
running 15000 at 11:28:41
running 15500 at 11:28:51
running 16000 at 11:29:03
running 16500 at 11:29:14
running 17000 at 11:29:25
running 17500 at 11:29:36
running 18000 at 11:29:47
running 18500 at 11:29:58
running 19000 at 11:30:09
running 19500 at 11:30:21
runnin

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:56<00:00,  2.36s/it]


running 500 at 11:57:59
running 1000 at 11:57:59
running 1500 at 11:58:00
running 2000 at 11:58:01
running 2500 at 11:58:03
running 3000 at 11:58:04
running 3500 at 11:58:06
running 4000 at 11:58:08
running 4500 at 11:58:11
running 5000 at 11:58:13
running 5500 at 11:58:17
running 6000 at 11:58:20
running 6500 at 11:58:24
running 7000 at 11:58:28
running 7500 at 11:58:32
running 8000 at 11:58:37
running 8500 at 11:58:42
running 9000 at 11:58:47
running 9500 at 11:58:53
running 10000 at 11:58:59
running 10500 at 11:59:05
running 11000 at 11:59:12
running 11500 at 11:59:19
running 12000 at 11:59:27
running 12500 at 11:59:34
running 13000 at 11:59:42
running 13500 at 11:59:51
running 14000 at 11:59:59
running 14500 at 12:00:09
running 15000 at 12:00:18
running 15500 at 12:00:27
running 16000 at 12:00:38
running 16500 at 12:00:48
running 17000 at 12:00:58
running 17500 at 12:01:09
running 18000 at 12:01:20
running 18500 at 12:01:31
running 19000 at 12:01:43
running 19500 at 12:01:55
runnin

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:18<00:00,  2.59s/it]


running 500 at 12:31:46
running 1000 at 12:31:46
running 1500 at 12:31:47
running 2000 at 12:31:48
running 2500 at 12:31:49
running 3000 at 12:31:51
running 3500 at 12:31:53
running 4000 at 12:31:55
running 4500 at 12:31:58
running 5000 at 12:32:01
running 5500 at 12:32:04
running 6000 at 12:32:08
running 6500 at 12:32:12
running 7000 at 12:32:17
running 7500 at 12:32:22
running 8000 at 12:32:27
running 8500 at 12:32:32
running 9000 at 12:32:38
running 9500 at 12:32:44
running 10000 at 12:32:50
running 10500 at 12:32:56
running 11000 at 12:33:03
running 11500 at 12:33:10
running 12000 at 12:33:19
running 12500 at 12:33:26
running 13000 at 12:33:35
running 13500 at 12:33:44
running 14000 at 12:33:53
running 14500 at 12:34:03
running 15000 at 12:34:12
running 15500 at 12:34:23
running 16000 at 12:34:33
running 16500 at 12:34:45
running 17000 at 12:34:56
running 17500 at 12:35:07
running 18000 at 12:35:19
running 18500 at 12:35:32
running 19000 at 12:35:44
running 19500 at 12:35:57
runnin

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:39<00:00,  2.80s/it]


running 500 at 13:07:21
running 1000 at 13:07:22
running 1500 at 13:07:22
running 2000 at 13:07:24
running 2500 at 13:07:25
running 3000 at 13:07:27
running 3500 at 13:07:30
running 4000 at 13:07:32
running 4500 at 13:07:35
running 5000 at 13:07:38
running 5500 at 13:07:42
running 6000 at 13:07:46
running 6500 at 13:07:50
running 7000 at 13:07:54
running 7500 at 13:07:59
running 8000 at 13:08:05
running 8500 at 13:08:10
running 9000 at 13:08:16
running 9500 at 13:08:23
running 10000 at 13:08:30
running 10500 at 13:08:37
running 11000 at 13:08:44
running 11500 at 13:08:52
running 12000 at 13:09:01
running 12500 at 13:09:09
running 13000 at 13:09:18
running 13500 at 13:09:27
running 14000 at 13:09:38
running 14500 at 13:09:48
running 15000 at 13:09:58
running 15500 at 13:10:09
running 16000 at 13:10:20
running 16500 at 13:10:31
running 17000 at 13:10:43
running 17500 at 13:10:55
running 18000 at 13:11:08
running 18500 at 13:11:21
running 19000 at 13:11:34
running 19500 at 13:11:48
runnin